In [4]:
!pip install langchain
!pip install pypdf
!pip install sentence-transformers
!pip install accelerate
!pip install chromadb
!pip install "unstructured[all-docs]"



  Using cached langchain-0.1.0-py3-none-any.whl (797 kB)
  Using cached dataclasses_json-0.6.3-py3-none-any.whl (28 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langchain_community-0.0.9-py3-none-any.whl (1.5 MB)
  Using cached langchain_core-0.1.7-py3-none-any.whl (214 kB)
  Using cached langsmith-0.0.77-py3-none-any.whl (48 kB)
  Using cached marshmallow-3.20.1-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.0 MB/s eta 0:00:00
     

In [4]:
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from google.colab import files
import os

uploaded_file = files.upload()

# Get the file name
file_name = list(uploaded_file.keys())[0]

# Create a temporary directory
temp_dir = "temp"
os.makedirs(temp_dir, exist_ok=True)

# Save the uploaded file to the temporary directory
temp_path = os.path.join(temp_dir, file_name)
with open(temp_path, "wb") as temp_file:
    temp_file.write(uploaded_file[file_name])

# Process the uploaded file
loader = UnstructuredFileLoader(temp_path)
documents = loader.load()
for document in documents:
    print(document.page_content)
# We cant load the whole pdf into the program so we split the pdf into chunks
# We use RecursiveCharacterTextSplitter to split the pdf into chunks
# Each chunk is 500 characters long and the chunks overlap by 200 characters (You can change this according to your needs)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=400)
texts = text_splitter.split_documents(documents)

# We use SentenceTransformerEmbeddings to embed the text chunks
# Embeddings are used to find the similarity between the query and the text chunks
# We use multi-qa-mpnet-base-dot-v1 model to embed the text chunks
# We need to save the embeddings to disk so we use persist_directory to save the embeddings to disk
embeddings = SentenceTransformerEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")
persist_directory = "/content/chroma/"

# Chroma is used to store the embeddings
# We use from_documents to store the embeddings
# We use the persist_directory to save the embeddings to disk
db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)

# To save and load the saved vector db (if needed in the future)
# Persist the database to disk
# db.persist()
# db = Chroma(persist_directory="db", embedding_function=embeddings)

checkpoint = "MBZUAI/LaMini-Flan-T5-783M"

# Initialize the tokenizer and base model for text generation
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map="auto",
    torch_dtype=torch.float32
)

pipe = pipeline(
    'text2text-generation',
    model = base_model,
    tokenizer = tokenizer,
    max_length = 512,
    do_sample = True,
    temperature = 0.3,
    top_p= 0.95
)

# Initialize a local language model pipeline
local_llm = HuggingFacePipeline(pipeline=pipe)
# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type='stuff',
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
    return_source_documents=True,
)

while True:
  input_query = str(input("Enter your query:"))
  llm_response = qa_chain({"query": input_query})
  print(llm_response['result'])


Saving kech101-1-3.pdf to kech101-1-3 (1).pdf


Unit 1.indd 1

Unit 1

Some BaSic conceptS of chemiStry

After studying this unit, you will be able to • appreciate the contribution of India in the development of chemistry understand the role of chemistry in different spheres of life;

explain the characteristics of three states of matter;

classify different substances into elements, compounds and mixtures;

use scientific notations and determine significant figures; • differentiate between precision and accuracy;

define SI base units and convert physical quantities from one system of units to another;

Chemistry is the science of molecules and their transformations. It is the science not so much of the one hundred elements but of the infinite variety of molecules that may be built from them.

Roald Hoffmann

Science can be viewed as a continuing human effort to systematise knowledge for describing and understanding nature. You have learnt in your previous classes that we come across diverse substances present in nature and changes

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your query:matter is ultimately made of 


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The provided context does not provide information to answer the question about what matter is ultimately made of.
Enter your query:When did tanning of leather were practised according to RigVeda


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


According to Rigveda, tanning of leather and dying of cotton were practiced during 1000-400 BCE.


KeyboardInterrupt: Interrupted by user